In [33]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.sans-serif"]=["WenQuanYi Micro Hei"] #设置字体
plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题

# 禁止随机，结果可复现
random.seed(42)
np.random.seed(42)

## 超参数选择

In [34]:
emo_index = ['上证综合情绪值', '沪深300情绪值', '创业板情绪值'][0]
model = 'Deep-learning/BERT'

ROOT_PATH = '/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Temporal_Clustering'
Clustering_Method = 'Other_Clusters'
Emotion_Data_PATH = f'{ROOT_PATH}/data/Emotion_Data/{model}'   # 情绪数据路径
Financial_Data_PATH = f'{ROOT_PATH}/data/Financial_Data' # 金融数据路径

os.makedirs(f'{ROOT_PATH}/{Clustering_Method}/个股分析/基于密度的聚类/{emo_index}', exist_ok=True)

print(f"Running with: emo_index={emo_index}, model={model}")

Running with: emo_index=上证综合情绪值, model=Deep-learning/BERT


## 数据准备

In [35]:
"""读取股吧个股的数据"""
all_data = []
file_list = [f for f in os.listdir(Emotion_Data_PATH) if f.endswith('.csv')]

for file in file_list:
    file_path = os.path.join(Emotion_Data_PATH, file)
    df = pd.read_csv(file_path)
    stock_code = os.path.splitext(file)[0] # 获取股票编号（文件名去掉扩展名）
    
    # 提取每一行的日期和情绪值
    for _, row in df.iterrows():
        new_row = {
            '股票编号': stock_code,
            '日期': row['日期'],
            '上证综合情绪值': row['上证综合情绪值'],
            '沪深300情绪值': row['沪深300情绪值'],
            '创业板情绪值': row['创业板情绪值']
        }
        all_data.append(new_row)
        
guba_data = pd.DataFrame(all_data)
guba_data

,股票编号,日期,上证综合情绪值,沪深300情绪值,创业板情绪值
0,601933,2024-12-27,0.007,-0.044,-0.015
1,601933,2024-11-18,-0.155,-0.236,-0.354
2,601933,2024-11-17,0.001,0.022,0.004
3,601933,2024-11-17,-0.346,-0.337,-0.407
4,601933,2024-11-17,-0.247,-0.309,-0.358
...,...,...,...,...,...
495326,601919,2021-04-09,0.253,0.327,0.457
495327,601919,2021-04-09,-0.177,-0.190,-0.196
495328,601919,2021-04-08,0.218,0.194,0.167
495329,601919,2021-04-08,0.148,0.142,0.346


In [36]:
"""读取股票回报率的数据"""
return_data = pd.read_csv(f'{Financial_Data_PATH}/日个股回报率.csv', dtype={'股票编号': str})
return_data

,股票编号,日期,交易量,收益率变化
0,000002,2021-06-01,60990961,-0.003745
1,000002,2021-06-02,85354506,0.006015
2,000002,2021-06-03,50594187,-0.003363
3,000002,2021-06-04,71422364,-0.012748
4,000002,2021-06-07,64745280,-0.014812
...,...,...,...,...
154877,688981,2024-11-20,58507495,-0.017071
154878,688981,2024-11-21,56197106,0.002358
154879,688981,2024-11-22,79240108,-0.050588
154880,688981,2024-11-25,76905909,-0.029402


In [37]:
# 进行左连接，guba_data 为主表
merged_data = pd.merge(guba_data, return_data[['股票编号', '日期', '交易量', '收益率变化']], 
                       on=['股票编号', '日期'], 
                       how='left')
merged_data = merged_data.dropna()
merged_data

,股票编号,日期,上证综合情绪值,沪深300情绪值,创业板情绪值,交易量,收益率变化
1,601933,2024-11-18,-0.155,-0.236,-0.354,310012824.0,-0.004608
11,601933,2024-11-15,-0.178,-0.191,-0.171,423271473.0,-0.064655
12,601933,2024-11-15,0.072,0.066,0.130,423271473.0,-0.064655
13,601933,2024-11-14,-0.019,0.027,0.068,372259052.0,-0.053061
14,601933,2024-11-14,-0.006,0.012,0.003,372259052.0,-0.053061
...,...,...,...,...,...,...,...
495148,601919,2021-06-01,0.265,0.327,0.476,268816321.0,0.045859
495149,601919,2021-06-01,-0.006,0.002,0.020,268816321.0,0.045859
495150,601919,2021-06-01,-0.034,-0.003,0.016,268816321.0,0.045859
495151,601919,2021-06-01,0.041,0.018,0.071,268816321.0,0.045859


## 数据预处理

In [38]:
# Step 1: 对情绪值列进行 Min-Max 标准化
def min_max_normalization(df, cols):
    for col in cols:
        min_val = df[col].min()
        max_val = df[col].max()
        df[col] = 2 * (df[col] - min_val) / (max_val - min_val) - 1
    return df

# Step 2: 对每个股票编号的数据进行标准化和按日期汇总
def process_data(df):
    df_processed = pd.DataFrame()  # 用于存储结果
    for stock_code, stock_data in df.groupby('股票编号'):
        # 对每个股票编号内的数据进行标准化
        stock_data = min_max_normalization(stock_data, ['上证综合情绪值', '沪深300情绪值', '创业板情绪值'])
        
        # 按日期汇总数据，同时保留股票编号
        stock_summary = stock_data.groupby('日期').agg({
            '股票编号': 'first',  # 保留股票编号（在同一日期内它是相同的，使用 'first'）
            '上证综合情绪值': 'mean',  # 上证综合情绪值按日期取均值
            '沪深300情绪值': 'mean',  # 沪深300情绪值按日期取均值
            '创业板情绪值': 'mean',  # 创业板情绪值按日期取均值
            '交易量': 'mean',  # 交易量按日期求和
            '收益率变化': 'mean'  # 收益率变化按日期取均值
        }).reset_index(drop=False)
        
        df_processed = pd.concat([df_processed, stock_summary], ignore_index=True)
    
    return df_processed

# 调用处理函数
final_data = process_data(merged_data)
final_data

,日期,股票编号,上证综合情绪值,沪深300情绪值,创业板情绪值,交易量,收益率变化
0,2021-06-01,000002,0.316338,0.267833,0.280789,60990961.0,-0.003745
1,2021-06-08,000002,0.347227,0.287914,0.300312,44676494.0,0.004626
2,2021-06-10,000002,0.325725,0.281754,0.289027,53800776.0,-0.010035
3,2021-06-11,000002,0.285945,0.224180,0.235237,75853738.0,-0.014035
4,2021-06-15,000002,0.314849,0.268572,0.283593,89915501.0,-0.020957
...,...,...,...,...,...,...,...
85724,2024-11-12,688981,0.360433,0.328320,0.235728,108866759.0,-0.036864
85725,2024-11-13,688981,0.269057,0.242652,0.142033,80759477.0,-0.019484
85726,2024-11-14,688981,0.286646,0.247683,0.160005,76194102.0,-0.022897
85727,2024-11-15,688981,0.244977,0.218178,0.099522,71066743.0,-0.020233


## 基于密度的聚类

In [39]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

selected_df = final_data[['日期', '股票编号', '上证综合情绪值', '收益率变化']].copy()

# 按照每只股票编号聚合：提取统计特征
def extract_features(group):
    return pd.Series({
        '情绪均值': group['上证综合情绪值'].mean(),
        '情绪标准差': group['上证综合情绪值'].std(),
        '情绪最大值': group['上证综合情绪值'].max(),
        '收益均值': group['收益率变化'].mean(),
        '收益标准差': group['收益率变化'].std(),
        '收益最大值': group['收益率变化'].max(),
    })

features_df = selected_df.groupby('股票编号').apply(extract_features).reset_index()
features_df

/tmp/ipykernel_2109407/385675846.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features_df = selected_df.groupby('股票编号').apply(extract_features).reset_index()


,股票编号,情绪均值,情绪标准差,情绪最大值,收益均值,收益标准差,收益最大值
0,000002,0.325232,0.051314,0.610222,-0.001376,0.025227,0.100437
1,000061,0.398932,0.123187,1.000000,0.000404,0.016299,0.079304
2,000063,0.421707,0.058419,0.657805,0.000499,0.025419,0.100161
3,000069,-0.169180,0.136894,1.000000,-0.000917,0.024606,0.101887
4,000100,-0.022380,0.078269,0.341941,0.000017,0.021839,0.100642
...,...,...,...,...,...,...,...
178,603986,0.165281,0.100334,0.500709,0.000741,0.031519,0.100047
179,688005,0.009524,0.096491,0.346847,0.001284,0.037750,0.182759
180,688027,0.054734,0.101542,0.468786,0.005550,0.042660,0.199337
181,688029,-0.274583,0.226751,1.000000,0.004512,0.027216,0.121161


In [40]:
# === 标准化特征 ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features_df.drop(columns=['股票编号']))

# === DBSCAN 聚类 ===
dbscan = DBSCAN(eps=0.8, min_samples=3)  # eps和min_samples需调参
labels = dbscan.fit_predict(X_scaled)

features_df['聚类标签'] = labels
features_df.rename(columns={'股票编号': 'Stkcd'}, inplace=True)

In [41]:
# ==== 4. 读取公司信息并拼接 ====
company_info_df = pd.read_csv(f'{Financial_Data_PATH}/中国公司代码.csv', dtype={'Stkcd': str})
features_df['Stkcd'] = features_df['Stkcd'].astype(str)

# 左连接拼接公司名称
merged_df = pd.merge(features_df[['Stkcd', '聚类标签']], company_info_df[['Stkcd', 'Stknme']], on='Stkcd', how='left')

# ==== 5. 保存最终结果 ====
output_path = f'{ROOT_PATH}/{Clustering_Method}/个股分析/基于密度的聚类/{emo_index}/clusters_results.csv'
merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
merged_df

,Stkcd,聚类标签,Stknme
0,000002,0,万科A
1,000061,-1,农产品
2,000063,0,中兴通讯
3,000069,-1,华侨城A
4,000100,0,TCL科技
...,...,...,...
178,603986,0,兆易创新
179,688005,-1,容百科技
180,688027,-1,国盾量子
181,688029,-1,南微医学
